# Image Grid Builder
Set parameters below, then run the next cell to generate a grid image from a folder.


In [6]:
from pathlib import Path

# --- Parameters ---
input_folder = Path("eval_phrases_7/phrase_p1(w46+w29)")  # <-- change this
output_file = Path("grid.jpg")
cols = 3              # 0 = auto
cell_width = 0        # 0 = auto
cell_height = 0       # 0 = auto
padding = 8
bg = "#ffffff"


In [7]:
import math
from PIL import Image

IMAGE_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".gif", ".tif", ".tiff", ".webp"}


def iter_images(folder: Path):
    for entry in sorted(folder.iterdir()):
        if entry.is_file() and entry.suffix.lower() in IMAGE_EXTS:
            yield entry


def load_and_fit(path: Path, cell_w: int, cell_h: int, bg_color):
    with Image.open(path) as im:
        im = im.convert("RGBA")
        scale = min(cell_w / im.width, cell_h / im.height)
        new_w = max(1, int(im.width * scale))
        new_h = max(1, int(im.height * scale))
        im = im.resize((new_w, new_h), Image.LANCZOS)
        canvas = Image.new("RGBA", (cell_w, cell_h), bg_color)
        off_x = (cell_w - new_w) // 2
        off_y = (cell_h - new_h) // 2
        canvas.paste(im, (off_x, off_y), im)
        return canvas.convert("RGB")


if not input_folder.is_dir():
    raise ValueError(f"Folder not found: {input_folder}")

images = list(iter_images(input_folder))
if not images:
    raise ValueError("No images found in the folder.")

sizes = []
for p in images:
    with Image.open(p) as im:
        sizes.append(im.size)

max_w = max(w for w, _ in sizes)
max_h = max(h for _, h in sizes)
cell_w = cell_width or max_w
cell_h = cell_height or max_h

cols_final = cols or math.ceil(math.sqrt(len(images)))
rows = math.ceil(len(images) / cols_final)

grid_w = cols_final * cell_w + (cols_final - 1) * padding
grid_h = rows * cell_h + (rows - 1) * padding

grid = Image.new("RGB", (grid_w, grid_h), bg)

for idx, img_path in enumerate(images):
    r = idx // cols_final
    c = idx % cols_final
    x = c * (cell_w + padding)
    y = r * (cell_h + padding)
    cell = load_and_fit(img_path, cell_w, cell_h, bg)
    grid.paste(cell, (x, y))

grid.save(output_file)
print(f"Wrote {output_file}")


Wrote grid.jpg
